In [2]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import time
from tqdm import tqdm

# Load and preprocess data
def prepare_data(pca_components):
    data = pd.read_csv('data-30s.csv')
    X = data.iloc[:, :-1]  # All features except last column
    y = data.iloc[:, -1]   # Last column
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply PCA
    pca = PCA(n_components=pca_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    return train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

# Quantum circuit for amplitude embedding
def create_quantum_circuit(n_qubits):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def quantum_circuit(inputs):
        # Amplitude embedding
        qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)
        # Measure all qubits
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    
    return quantum_circuit

# Quantum feature mapping
def quantum_feature_mapping(X, n_qubits):
    quantum_circuit = create_quantum_circuit(n_qubits)
    quantum_features = []
    
    for sample in tqdm(X, desc="Quantum Processing"):
        # Pad the input if necessary to match 2^n_qubits
        padded_input = np.pad(sample, (0, 2**n_qubits - len(sample)), 'constant')
        # Normalize the input
        normalized_input = padded_input / np.linalg.norm(padded_input)
        quantum_features.append(quantum_circuit(normalized_input))
        
    return np.array(quantum_features)

def evaluate_classifier(clf, X_train, X_test, y_train, y_test):
    # Training time
    start_time = time.time()
    clf.fit(X_train, y_train)
    fit_time = time.time() - start_time
    
    # Prediction time
    start_time = time.time()
    y_pred = clf.predict(X_test)
    predict_time = time.time() - start_time
    
    # Calculate metrics
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_pred),
        'Kappa': cohen_kappa_score(y_test, y_pred),
        'Fit Time': fit_time,
        'Predict Time': predict_time
    }
    return metrics



In [3]:
def main():
    classifiers = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'ExtraTrees': ExtraTreesClassifier(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, random_state=42)
    'CatBoost': CatBoostClassifier(random_state=42, verbose=False),
    'AdaBoost': AdaBoostClassifier(random_state=42),    
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
    }
    
    pca_components = [2, 10, 15]
    results = []
    
    for n_components in pca_components:
        print(f"\nProcessing PCA components: {n_components}")
        X_train, X_test, y_train, y_test = prepare_data(n_components)
        
        # Calculate required qubits
        n_qubits = int(np.ceil(np.log2(n_components)))
        
        # Get quantum features
        X_train_quantum = quantum_feature_mapping(X_train, n_qubits)
        X_test_quantum = quantum_feature_mapping(X_test, n_qubits)
        
        for clf_name, clf in classifiers.items():
            # Classical features
            classical_metrics = evaluate_classifier(clf, X_train, X_test, y_train, y_test)
            classical_metrics.update({'Method': 'Classical', 'Classifier': clf_name, 'PCA': n_components})
            results.append(classical_metrics)
            
            # Quantum features
            quantum_metrics = evaluate_classifier(clf, X_train_quantum, X_test_quantum, y_train, y_test)
            quantum_metrics.update({'Method': 'Quantum', 'Classifier': clf_name, 'PCA': n_components})
            results.append(quantum_metrics)
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    print("\nResults:")
    print(results_df)
    
    # Save results
    results_df.to_csv('classification_results.csv', index=False)

if __name__ == "__main__":
    main()



Processing PCA components: 2


Quantum Processing: 100%|█████████████████████████████████████████████████████████| 2931/2931 [00:04<00:00, 731.15it/s]



Processing PCA components: 10


Quantum Processing: 100%|█████████████████████████████████████████████████████████| 2931/2931 [00:07<00:00, 394.63it/s]



Processing PCA components: 15


Quantum Processing: 100%|█████████████████████████████████████████████████████████| 2931/2931 [00:07<00:00, 414.87it/s]



Results:
    Accuracy  Precision    Recall        F1   ROC AUC     Kappa   Fit Time  \
0   0.760491   0.754037  0.798684  0.775719  0.759016  0.519220   1.062939   
1   0.629819   0.635345  0.671711  0.653022  0.628201  0.256970   1.054865   
2   0.743773   0.734594  0.792105  0.762267  0.741907  0.485281   0.182064   
3   0.640737   0.638576  0.707895  0.671451  0.638143  0.277449   0.125000   
4   0.766974   0.755651  0.813816  0.783655  0.765164  0.531910   6.688555   
5   0.632207   0.636927  0.676316  0.656031  0.630504  0.261632  14.073542   
6   0.842033   0.829694  0.875000  0.851745  0.840760  0.682961  10.297431   
7   0.817127   0.807116  0.850658  0.828315  0.815832  0.632983   4.332966   
8   0.825316   0.817781  0.853289  0.835158  0.824235  0.649560   0.233788   
9   0.787445   0.773642  0.834211  0.802786  0.785638  0.573009   0.156254   
10  0.844763   0.829721  0.881579  0.854864  0.843341  0.688343  15.643151   
11  0.806551   0.795046  0.844737  0.819139  0.805076 